## Data Preparation, Model Design, Training, and Evaluation

In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Load the dataset
with open("Sequential_Dataset.txt", "r") as file:
    text_data = file.read()

# Create character-to-index and index-to-character mappings
chars = sorted(list(set(text_data)))
char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))

SEQUENCE_LENGTH = 20
inputs = []
outputs = []

for i in range(0, len(text_data) - SEQUENCE_LENGTH):
    sequence_in = text_data[i:i + SEQUENCE_LENGTH]
    sequence_out = text_data[i + SEQUENCE_LENGTH]
    inputs.append([char_to_index[char] for char in sequence_in])
    outputs.append(char_to_index[sequence_out])

num_chars = len(chars)

# Convert inputs and outputs for model
X = np.reshape(inputs, (len(inputs), SEQUENCE_LENGTH, 1))
X = X / float(num_chars)  # Normalize
y = to_categorical(outputs)

# LSTM Model Design
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(num_chars, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model
model.fit(X, y, epochs=15, batch_size=150)

# Predict the next 5 characters for the 5th training record
initial_sequence = text_data[4:24]
print("Initial Sequence: ", initial_sequence)
for i in range(5):
    sequence = [char_to_index[char] for char in initial_sequence]
    sequence = np.reshape(sequence, (1, len(sequence), 1))
    sequence = sequence / float(num_chars)
    prediction = model.predict(sequence, verbose=0)
    index = np.argmax(prediction)
    initial_sequence += index_to_char[index]
    initial_sequence = initial_sequence[1:]
print("Predicted Sequence: ", initial_sequence)

# Hyperparameter tuning using grid search
def create_model(dropout_rate=0.2, lstm_units=256, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(lstm_units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_chars, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

model = KerasClassifier(build_fn=create_model)

# Define the grid search parameters
batch_size = [150, 200]
epochs = [15, 20]
dropout_rate = [0.1, 0.2]
lstm_units = [256, 512]
optimizer = ['SGD', 'Adam']

param_grid = dict(batch_size=batch_size, epochs=epochs, dropout_rate=dropout_rate, lstm_units=lstm_units, optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, y)

# Print the best hyperparameters
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")



## Hyperparameter Tuning:

In the above code, we used grid search to find the optimal hyperparameters for our LSTM model. The parameters we tuned include:
- Batch size
- Epochs
- Dropout rate
- LSTM units
- Optimizer

The results from the grid search provide the best combination of these hyperparameters for the highest accuracy on the validation set.

## Presentation and Documentation:

The notebook provides a structured approach to the task of character generation using LSTM. The code is commented for clarity, and each section is demarcated for better understanding.

For a detailed report:
1. **Introduction**: Brief overview of the project and objectives.
2. **Data Exploration**: Characteristics of the dataset.
3. **Model Design and Training**: About the chosen model architecture and training process.
4. **Hyperparameter Tuning**: Method used for tuning and best parameters obtained.
5. **Results**: Display the initial 10 training sequences and their respective outputs. Show the 5-character prediction for the 5th training record.
6. **Conclusion**: Summary of findings, challenges faced during the project, lessons learned, and future recommendations.
